By: Violeta, Helena, Javier y Laso

---

In [1]:
import nipype.interfaces.spm as spm         # the spm interfaces
import nipype.pipeline.engine as pe         # the workflow and node wrappers
import nipype.interfaces.io as nio   
import nipype.pipeline.engine as pe
from nipype import Node, JoinNode, Workflow
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.utility import Merge

200225-09:30:38,302 nipype.utils INFO:
	 No new version available.


In [2]:
import fnmatch
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces import fsl
from nipype.testing import example_data
from nipype.interfaces import fsl
from nipype.testing import example_data
from nipype.interfaces.fsl import Merge as Merge_fsl
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP
from nipype.interfaces import fsl
from nipype.interfaces.fsl import Eddy
import shutil
import os
sep=os.path.sep
import json
from datetime import datetime

main= sep+'Users'+sep+'administrator'+sep+'Documents'+sep+'nifti' 

In [3]:
dti = pe.Workflow(name='dti')
dti.base_dir = '.'

In [4]:
sb='sub-sp01'
ses='ses-02'

roi1 = pe.Node(interface=ExtractROI(t_min=0,t_size=1), name='roi1')
roi1.inputs.in_file=main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.nii'
roi1.inputs.roi_file= main+sep+sb+sep+ses+sep+'dwi'+sep+'nodif_'+sb+ses+'-dwi.nii.gz'

In [5]:
roi2 = pe.Node(interface=ExtractROI(t_min=0,t_size=1), name='roi2')
roi2.inputs.in_file=main+sep+sb+sep+ses+sep+'fmap'+sep+sb+ses+'_acq-dwi_dir-1_epi.nii'
roi2.inputs.roi_file= main+sep+sb+sep+ses+sep+'fmap'+sep+'nodif_'+sb+ses+'-fmap.nii.gz'

In [6]:
flirt = pe.Node(interface=fsl.FLIRT(bins=640, cost_func='mutualinfo'), name='flirt')
#flirt.inputs.in_file = main+sep+sb+sep+ses+sep+'dwi'+sep+'nodif_'+sb+ses+'-dwi.nii.gz'
#flirt.inputs.reference = main+sep+sb+sep+ses+sep+'dwi'+sep+'nodif_'+sb+ses+'-dwi.nii.gz'
flirt.inputs.output_type = "NIFTI_GZ"

In [7]:
#dti.connect(roi1, 'roi_file', flirt, 'in_file')
dti.connect([(roi2, flirt, [(('roi_file'),
                            'in_file')]),])

In [8]:
#dti.connect(roi1, 'roi_file', flirt, 'in_file')
dti.connect([(roi1, flirt, [(('roi_file'),
                            'reference')]),])

In [9]:
from nipype.interfaces.utility import Merge as Merge_other
mi = pe.Node(interface=Merge_other(2), name='mi')
dti.connect(roi1,'roi_file',mi,'in1')
dti.connect(flirt,'out_file',mi,'in2')

In [10]:
merg= pe.Node(interface=Merge_fsl(), name='merg')
dti.connect(mi,'out',merg,'in_files')
merg.inputs.dimension = 't'
merg.inputs.output_type = 'NIFTI_GZ'

In [11]:
def json_params(files,save_dir,name,ckpt=0):
    texts=[]
    for file in files:
        with open(file) as f:
            d = json.load(f)
            AT=(d['AcquisitionMatrixPE'])
            ES=d['EffectiveEchoSpacing']
            j_direction=d['PhaseEncodingDirection']
            ReadOut_Time =AT*ES*0.5
            text= j_direction.replace('j-','0 -1 0 ').replace('j','0 1 0 ').replace('i-','-1 0 0 ').replace('i','1 0 0 ').replace('k-','0 0 -1 ').replace('k','0 0 1')+ str(ReadOut_Time)
            texts.append(text)
        with open(os.path.join(save_dir, name), 'w') as f:
            for item in texts:
                f.write("%s\n" % item)
    if ckpt:
        print('file saved as: ', save_dir+sep+name)
        print('content saved: ', texts)

directory_dwi=main+sep+sb+sep+ses+sep+'dwi'
directory_fmap=main+sep+sb+sep+ses+sep+'fmap'
params_files=[directory_dwi+sep+sb+ses+'_dir-AP_dwi.json',directory_fmap+sep+sb+ses+'_acq-dwi_dir-1_epi.json']
json_params(params_files,main,'acq_params-'+sb+ses+'.txt')

In [12]:
tpup=pe.Node(interface=TOPUP(),name='tpup')
dti.connect(merg,'merged_file',tpup,'in_file')
tpup.inputs.encoding_file = main+sep+'acq_params-'+sb+ses+'.txt'
tpup.inputs.output_type = "NIFTI_GZ"

---

In [13]:
mu = pe.Node(interface=Merge_other(2), name='mu')
dti.connect(roi1,'roi_file',mu,'in1')
dti.connect(flirt,'out_file',mu,'in2')

In [14]:
apptpup=pe.Node(interface=ApplyTOPUP(),name='apptpup')
dti.connect(mu,'out',apptpup,'in_files')
apptpup.inputs.encoding_file = main+sep+'acq_params-'+sb+ses+'.txt'
dti.connect(tpup,'out_fieldcoef',apptpup,'in_topup_fieldcoef')
dti.connect(tpup,'out_movpar',apptpup,'in_topup_movpar')
apptpup.inputs.output_type = "NIFTI_GZ"

In [15]:
bet=pe.Node(interface=fsl.BET(),name='bet')
dti.connect(apptpup,'out_corrected',bet,'in_file')
bet.inputs.frac = 0.2
bet.inputs.mask = True

---

In [16]:
def create_index(file,save_dir,name):
    #print("> creating index",print_time(),"      ", end="", flush=True)
    with open(file) as f:
        lasito=f.read()
        volumes= len(lasito.split(' '))
    with open(os.path.join(save_dir, name), 'w') as f:
        for i in range(volumes):
            f.write("1 ")
file= main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.bval'
save_dir=main
name='index-'+sb+ses+'_dir-AP_dwi.txt'
create_index(file,save_dir,name)

In [17]:
eddy=pe.Node(interface=Eddy(),name='eddy')
eddy.inputs.in_file = main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.nii'
dti.connect(bet,'mask_file',eddy,'in_mask')
eddy.inputs.in_index = main+sep+'index-'+sb+ses+'_dir-AP_dwi.txt'
eddy.inputs.in_acqp  = main+sep+'acq_params-'+sb+ses+'.txt'
eddy.inputs.in_bvec  = main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.bvec'
eddy.inputs.in_bval  = main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.bval'

In [18]:
datasink = pe.Node(interface=nio.DataSink(), name="datasink")
datasink.inputs.base_directory = os.path.abspath('./Results')

In [19]:
lasosink = pe.Node(interface=nio.DataSink(), name="lasosink")
lasosink.inputs.base_directory = os.path.abspath('./Results')

In [20]:
#dti.connect(tpup,'out_corrected',datasink,'topup')
dti.connect(apptpup,'out_corrected',datasink,'applytopup')

In [21]:
dti.connect(eddy,'out_corrected',lasosink,'eddy')

In [22]:
dti.write_graph(graph2use='colored')

200225-09:33:48,79 nipype.workflow INFO:
	 Generated workflow graph: ./dti/graph.png (graph2use=colored, simple_form=True).


'./dti/graph.png'

In [23]:
dti.run()

200225-09:34:22,163 nipype.workflow INFO:
	 Workflow dti settings: ['check', 'execution', 'logging', 'monitoring']
200225-09:34:22,224 nipype.workflow INFO:
	 Running serially.
200225-09:34:22,226 nipype.workflow INFO:
	 [Node] Setting-up "dti.roi1" in "/Users/administrator/Documents/nifti/dti/roi1".
200225-09:34:22,280 nipype.workflow INFO:
	 [Node] Cached "dti.roi1" - collecting precomputed outputs
200225-09:34:22,282 nipype.workflow INFO:
	 [Node] "dti.roi1" found cached.
200225-09:34:22,284 nipype.workflow INFO:
	 [Node] Setting-up "dti.roi2" in "/Users/administrator/Documents/nifti/dti/roi2".
200225-09:34:22,346 nipype.workflow INFO:
	 [Node] Cached "dti.roi2" - collecting precomputed outputs
200225-09:34:22,347 nipype.workflow INFO:
	 [Node] "dti.roi2" found cached.
200225-09:34:22,349 nipype.workflow INFO:
	 [Node] Setting-up "dti.flirt" in "/Users/administrator/Documents/nifti/dti/flirt".
200225-09:34:22,382 nipype.workflow INFO:
	 [Node] Cached "dti.flirt" - collecting precomp

200225-10:21:41,107 nipype.workflow INFO:
	 ***********************************
200225-10:21:41,109 nipype.workflow ERROR:
	 could not run node: dti.eddy
200225-10:21:41,110 nipype.workflow INFO:
	 crashfile: /Users/administrator/Documents/nifti/crash-20200225-102141-administrator-eddy-2d738b39-90f2-4f77-b856-ffb741a253e3.pklz
200225-10:21:41,112 nipype.workflow INFO:
	 ***********************************


RuntimeError: Workflow did not execute cleanly. Check log for details

---

checkpoint at 20:20 --> topup working